<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env:gnn-pytorch]</code> for this notebook. </div>

# 3. Inference - Tracking

In [1]:
import os
import subprocess
import sys
import re

In [3]:
software_dir = 'software'
sys.path.append(software_dir)

from inference_clean import predict

inference_dir = 'cell-tracker-gnn/src/inference'
data_dir = 'data'

models = [
    'Fluo-N2DL-HeLa',
    'Fluo-N2DH-SIM+'
]

pattern = re.compile('\d{3}')
data_ids = [f for f in os.listdir(os.path.join(data_dir, models[0])) if pattern.fullmatch(f)]

/opt/conda/envs/cell-tracking-challenge/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
modality = '2D'
min_size = '10'

model_metric_learning_dir = os.path.join(software_dir, 'parameters/Features_Models')
modesl_pytorch_lightning = {
    'Fluo-N2DL-HeLa': os.path.join(software_dir, 'parameters/Tracking_Models/Fluo-N2DL-HeLa/checkpoints/epoch=312.ckpt'),
    'Fluo-N2DH-SIM+': os.path.join(software_dir, 'parameters/Tracking_Models/Fluo-N2DH-SIM+/checkpoints/epoch=132.ckpt')
}

In [ ]:
process_output = []
for m in models:
    print('------', m, '------')
    for i in data_ids:
        print(i)
        p = subprocess.run(
            ['python', os.path.join(software_dir, 'preprocess_seq2graph_clean.py'),
             '-cs', min_size,
             '-ii', os.path.join(data_dir, m, i),
             '-iseg', os.path.join(data_dir, m, f'{i}_SEG_RES'),
             '-im', os.path.join(model_metric_learning_dir, m, 'all_params.pth'),
             '-oc', os.path.join(data_dir, m, f'{i}_CSV')
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
        process_output.append(p)

        p = subprocess.run(
            ['python', os.path.join(software_dir, 'inference_clean.py'),
             '-mp', models_pytorch_lightning[m],
             '-ns', i,
             '-oc', os.path.join(data_dir, m)
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
        process_output.append(p)

        p = subprocess.run(
            ['python', os.path.join(inference_dir, 'postprocess_clean.py'),
             '-modality', modality,
             '-iseg', os.path.join(data_dir, m, f'{i}_SEG_RES'),
             '-oi', os.path.join(data_dir, m, f'{i}_RES_inference')
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)
        process_output.append(p)